## M04 Live Coding

In [54]:
import numpy as np
import pandas as pd
import dotenv 
import requests
import json
import os

In [55]:
# News API 

In [56]:
# User - agent string

r = requests.get('https://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']
headers = {'User-agent': useragent}

In [57]:
# Load ENV

dotenv.load_dotenv()
newskey = os.getenv('newskey')

In [58]:
params = {'apikey': newskey,
         'q': 'Bob Good'}

url = 'https://newsapi.org/v2/everything'
r = requests.get(url, headers = headers, params = params)

In [59]:
myjson = json.loads(r.text)
print(myjson)
news_df = pd.json_normalize(myjson, record_path = ['articles'])

{'status': 'ok', 'totalResults': 2058, 'articles': [{'source': {'id': 'business-insider', 'name': 'Business Insider'}, 'author': 'Brent D. Griffiths', 'title': 'Trump continues his revenge tour against pro-DeSantis lawmakers', 'description': "Donald Trump hasn't forgotten that Rep. Bob Good of Virginia initially backed Ron DeSantis over him.", 'url': 'https://www.businessinsider.com/trump-revenge-desantis-primary-challengers-bob-good-virginia-2024-2024-5', 'urlToImage': 'https://i.insider.com/6655e798a961b37edf3a3db1?width=1200&format=jpeg', 'publishedAt': '2024-05-28T14:47:14Z', 'content': "Rep. Bob Good of Virginia had tried to appear close to former President Donald Trump before Trump backed Good's primary foe.Anna Moneymaker/Getty Images\r\n<ul><li>Donald Trump is backing a primary cha… [+2938 chars]"}, {'source': {'id': 'the-hill', 'name': 'The Hill'}, 'author': 'Julia Manchester', 'title': 'MAGA civil war reaches climax in Bob Good primary', 'description': 'The MAGA civil war sur

In [60]:
## Congress API

congresskey = os.getenv('congresskey')

In [61]:
root = 'https://api.congress.gov/v3'
endpoint = '/member'
params = {'api_key': congresskey, 'format': 'json', 'currentMember': 'true', 'limit': 30}
r = requests.get(root + endpoint, params = params, headers = headers)
r

<Response [200]>

In [62]:
members_df = pd.json_normalize(json.loads(r.text), record_path = ['members'])

In [63]:
bob_good_info = members_df.query('state=="Virginia" & district==5')
bobgood_id = bob_good_info['bioguideId'][15]

In [64]:
### Sponsored Legislation

In [65]:
endpoint = f'/member/{bobgood_id}/sponsored-legislation'

In [66]:
r = requests.get(root + endpoint, params = params, headers = headers)
r

<Response [200]>

In [67]:
bobgood_bills = pd.json_normalize(json.loads(r.text), record_path = 'sponsoredLegislation')
#bobgood_bills

In [68]:
[print(x) for x in bobgood_bills['title']]

Protecting Student Athletes’ Economic Freedom Act
Protect Our Police Act
Silencers Help Us Save Hearing Act
Apprenticeship Freedom Act
To prohibit Federal funding of National Public Radio and the use of Federal funds to acquire radio content.
Iran China Accountability Act of 2024
Every Town A Border Town Act
Denouncing calls for a cease-fire in Gaza.
Blue Ridge Fire Safety Act
To prohibit the Administrator of the Environmental Protection Agency from finalizing, implementing, or enforcing the proposed rule related to revisions to the air emissions reporting requirements, and for other purposes.
No Regulation Through Litigation Act of 2023
Veterans Education is Timeless Act of 2023
No American Climate Corps Act
Expressing support for the designation of October 12, 2023, as "National Loggers Day".
No Discrimination in My Benefits Act
No Vaccine Mandates in Higher Education Act
Empowering Local Curriculum Act
Empowering Parents Act
Defending Students’ Civil Rights Act of 2023
Preserving Sa

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [69]:
# Open Secrets

In [81]:
opensecretskey = os.getenv('opensecretskey')

root = 'https://www.opensecrets.org/api'
endpoint = '/?method=getLegislators'

params = {'apikey': opensecretskey, 'id': 'VA', 'output': 'json'}

r = requests.get(root + endpoint, params = params, headers = headers)


In [82]:
member_df = pd.json_normalize(json.loads(r.text), record_path=['response', 'legislator'])

In [83]:
member_df.columns = [x.replace('@attributes.', '') for x in member_df.columns]

In [84]:
member_df

,cid,firstlast,lastname,party,office,gender,first_elected,exit_code,comments,phone,...,website,webform,congress_office,bioguide_id,votesmart_id,feccandid,twitter_id,youtube_url,facebook_id,birthdate
0,N00029459,Rob Wittman,Wittman,R,VA01,M,2007,0,,202-225-4261,...,http://wittman.house.gov,https://wittman.house.gov/contact-form/,2055 Rayburn House Office Building,W000804,58133,H8VA01147,RobWittman,https://youtube.com/RobWittman,RepRobWittman,1959-02-03
1,N00048315,Jen Kiggans,Kiggans,R,VA02,F,2022,0,,,...,,,,K000399,,H2VA02064,,,,
2,N00002147,Bobby Scott,Scott,D,VA03,M,1992,0,,202-225-8351,...,http://bobbyscott.house.gov,https://bobbyscott.house.gov/email-bobby-scott,1201 Longworth House Office Building,S000185,27117,H6VA01117,BobbyScott,https://youtube.com/repbobbyscott,RepBobbyScott,1947-04-30
3,N00051961,Jennifer McClellan,McClellan,D,VA04,F,2023,0,Elected in special to replace McEachin,,...,,,,,,,,,,
4,N00045557,Bob Good,Good,R,VA05,M,2020,0,,202225-4711,...,https://good.house.gov/,,"1213 Longworth House Office Building, Washingt...",456853,,,@RepBobGood,,,09/11/65
5,N00042296,Ben Cline,Cline,R,VA06,M,2018,0,,202-225-5431,...,https://cline.house.gov/,,1009 Longworth House Office Building,C001118,,H8VA06104,RepBenCline,,,1972-02-29
6,N00041418,Abigail Spanberger,Spanberger,D,VA07,F,2018,16,Running for Governor,202-225-2815,...,https://spanberger.house.gov/,,1239 Longworth House Office Building,S001209,,H8VA07094,RepSpanberger,,,1979-08-07
7,N00036018,Don Beyer,Beyer,D,VA08,M,2014,0,,202-225-4376,...,http://beyer.house.gov,,1119 Longworth House Office Building,B001292,,H4VA08224,repdonbeyer,,repdonbeyer,1950-06-20
8,N00032029,Morgan Griffith,Griffith,R,VA09,M,2010,0,,202-225-3861,...,http://morgangriffith.house.gov,https://morgangriffith.house.gov/contact/conta...,2202 Rayburn House Office Building,G000568,5148,H0VA09055,RepMGriffith,https://youtube.com/RepMorganGriffith,RepMorganGriffith,1958-03-15
9,N00041002,Jennifer Wexton,Wexton,D,VA10,F,2018,4,Retiring at end of 118th Congress,202-225-5136,...,https://wexton.house.gov/,,1217 Longworth House Office Building,W000825,,H8VA10106,RepWexton,,,1968-05-27


In [85]:
good_cid = member_df.query('lastname=="Good"')['cid'].tolist()[0]
good_cid

'N00045557'

In [91]:
endpoint = '/?method=candContrib'
params = {'apikey': opensecretskey, 'cid': good_cid, 'output': 'json'}
r = requests.get(root+endpoint, params = params, headers = headers)

In [92]:
r

<Response [200]>

In [93]:
json.loads(r.text)

{'response': {'contributors': {'@attributes': {'cand_name': 'Bob Good (R)',
    'cid': 'N00045557',
    'cycle': '2020',
    'origin': 'OpenSecrets',
    'source': 'https://www.opensecrets.org/members-of-congress/contributors?cid=N00045557&cycle=2020',
    'notice': "The organizations themselves did not donate, rather the money came from the organization's PAC, its individual members or employees or owners, and those individuals' immediate families."},
   'contributor': [{'@attributes': {'org_name': 'RST Marketing',
      'total': '15200',
      'pacs': '0',
      'indivs': '15200'}},
    {'@attributes': {'org_name': 'Liberty University',
      'total': '14567',
      'pacs': '0',
      'indivs': '14567'}},
    {'@attributes': {'org_name': 'Amelia Overhead Doors',
      'total': '11200',
      'pacs': '0',
      'indivs': '11200'}},
    {'@attributes': {'org_name': 'Law Offices Hiepler & Hiepler',
      'total': '11200',
      'pacs': '0',
      'indivs': '11200'}},
    {'@attributes':

In [99]:
r.text

'{"response":{"contributors":{"@attributes":{"cand_name":"Bob Good (R)","cid":"N00045557","cycle":"2020","origin":"OpenSecrets","source":"https:\\/\\/www.opensecrets.org\\/members-of-congress\\/contributors?cid=N00045557&cycle=2020","notice":"The organizations themselves did not donate, rather the money came from the organization\'s PAC, its individual members or employees or owners, and those individuals\' immediate families."},"contributor":[{"@attributes":{"org_name":"RST Marketing","total":"15200","pacs":"0","indivs":"15200"}},{"@attributes":{"org_name":"Liberty University","total":"14567","pacs":"0","indivs":"14567"}},{"@attributes":{"org_name":"Amelia Overhead Doors","total":"11200","pacs":"0","indivs":"11200"}},{"@attributes":{"org_name":"Law Offices Hiepler & Hiepler","total":"11200","pacs":"0","indivs":"11200"}},{"@attributes":{"org_name":"Justice Retail Operations","total":"10800","pacs":"0","indivs":"10800"}},{"@attributes":{"org_name":"Virginia Wagyu Farm","total":"8400","p

In [100]:
funding_df = pd.json_normalize(json.loads(r.text), record_path=['response','contributors','contributor'])

In [101]:
funding_df

,@attributes.org_name,@attributes.total,@attributes.pacs,@attributes.indivs
0,RST Marketing,15200,0,15200
1,Liberty University,14567,0,14567
2,Amelia Overhead Doors,11200,0,11200
3,Law Offices Hiepler & Hiepler,11200,0,11200
4,Justice Retail Operations,10800,0,10800
5,Virginia Wagyu Farm,8400,0,8400
6,Ariel Corp,5600,0,5600
7,Ashley Grace Bridal,5600,0,5600
8,Byrd Enterprises Unlimited,5600,0,5600
9,CSM Inc,5600,0,5600
